In [72]:
# Importing packages
import os
import pandas as pd
import numpy as np
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins.metadata.visualizers import tabulate

from qiime2.plugins import feature_classifier
from qiime2.plugins import metadata
from qiime2.plugins import taxa

from qiime2.plugins.taxa.methods import collapse
from qiime2.plugins.taxa.methods import filter_table
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.methods import relative_frequency

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm

from  statannot  import  add_stat_annotation

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set_theme(style="ticks", palette="pastel")

In [3]:
# Parameters
experiment_name = "miliane-CxAC-trim"
base_dir = "/mnt/nupeb/rede-micro/redemicro-miliane-nutri"
manifest_file = "/mnt/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/manifest/miliane-manifest-CxAC.csv"
metadata_file = "/mnt/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/metadata/miliane-metadata-CxAC.tsv"
class_col = "group-id"

In [20]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))
sheet_folder = os.path.abspath(os.path.join(experiment_folder, 'sheets'))
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')
# Input - Metataxonomy Artifacts
metatax_path = os.path.join(qiime_folder, 'metatax.qza')
# Input - Excel file
excel_path = os.path.join(sheet_folder, 'absolute-all_frequencies.xlsx')

In [36]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)
metadata_df = metadata_qa.to_dataframe()
# Load FeatureTable[Taxonomy]
metatax_qza = Artifact.load(metatax_path)
metatax_df = metatax_qza.view(Metadata).to_dataframe()
#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)
tabs_df = tabs.view(Metadata).to_dataframe().T


# Abundance of each species

In [11]:
tax_level = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']

In [70]:
# Load sheet
sheet_df = pd.read_excel(excel_path, sheet_name=tax_level[1])
# Normalize to relative abundance
sheet_df.iloc[:, 1:] = sheet_df.iloc[:, 1:] / sheet_df.iloc[:, 1:].sum(axis=0)
sheet_df

,Unnamed: 0,total_sum,AC,C,210421121711,210421121712,210421121713,210421121714,210421121715,210421121716,210421121717,210421121718,210421121719,210421121720,210421121721,210421121722
0,Bacteroidota,0.538830,0.592672,0.487726,0.374718,0.826962,0.465965,0.927498,0.301871,0.455986,0.614844,0.620039,0.516281,0.893761,0.573483,0.234413
1,Firmicutes,0.429183,0.364132,0.490928,0.608898,0.134290,0.528487,0.013479,0.684336,0.525888,0.375259,0.335527,0.462193,0.053294,0.404242,0.629668
2,Proteobacteria,0.011580,0.016064,0.007323,0.016383,0.010298,0.005404,0.014443,0.003271,0.008536,0.006559,0.036965,0.009057,0.006724,0.007556,0.049517
3,Cyanobacteria,0.008119,0.006687,0.009479,0.000000,0.014372,0.000000,0.038960,0.008036,0.001143,0.000786,0.001108,0.004380,0.017071,0.007846,0.001118
4,Deferribacterota,0.005819,0.011542,0.000386,0.000000,0.000000,0.000000,0.000000,0.001032,0.000000,0.000000,0.000000,0.000351,0.000000,0.004052,0.081399
5,Patescibacteria,0.005211,0.007072,0.003445,0.000000,0.014077,0.000000,0.002933,0.000863,0.007865,0.001375,0.004249,0.007080,0.026809,0.000982,0.001201
6,Verrucomicrobiota,0.000920,0.001420,0.000446,0.000000,0.000000,0.000000,0.002687,0.000254,0.000000,0.001178,0.001968,0.000234,0.002120,0.001385,0.001507
7,Actinobacteriota,0.000199,0.000125,0.000268,0.000000,0.000000,0.000144,0.000000,0.000336,0.000582,0.000000,0.000143,0.000074,0.000174,0.000077,0.000330
8,unclassified,0.000106,0.000218,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000047,0.000376,0.000695
9,Campilobacterota,0.000024,0.000049,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000351,0.000000,0.000000,0.000000


In [71]:
df1 = sheet_df.iloc[1, 4:].astype(float)
df2 = metadata_df['acido-butirico']
print(f"df1: {df1.shape} | df2: {df2.shape}")

df1: (12,) | df2: (12,)


In [72]:
sm

-0.06450862848211329